In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
import tensorflow.compat.v1 as tf
import numpy as np
tf.disable_eager_execution()

from sklearn.datasets import make_moons
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.datasets import fetch_california_housing
from tensorflow.python.ops import math_ops

from tensorflow.keras import layers

C:\Users\user\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\user\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\user\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\user\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [10]:
# create input function
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"
CSV_COLUMN_NAMES = ['SepalLength','SepalWidth',
                   'PetalLength','PetalWidth','Species']
SPECIES = ['Setosa','Versocolor','Virginica']

def load_data():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1],TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1],TEST_URL)
    
    train = pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
    X_train, y_train = train,train.pop('Species')
    
    test = pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
    X_test, y_test = test,test.pop('Species')
    
    return (X_train,y_train), (X_test,y_test)

(X_train,y_train), (X_test,y_test) = load_data()
print (X_train)

     SepalLength  SepalWidth  PetalLength  PetalWidth
0            6.4         2.8          5.6         2.2
1            5.0         2.3          3.3         1.0
2            4.9         2.5          4.5         1.7
3            4.9         3.1          1.5         0.1
4            5.7         3.8          1.7         0.3
..           ...         ...          ...         ...
115          5.5         2.6          4.4         1.2
116          5.7         3.0          4.2         1.2
117          4.4         2.9          1.4         0.2
118          4.8         3.0          1.4         0.1
119          5.5         2.4          3.7         1.0

[120 rows x 4 columns]


In [11]:
def train_input_fn(features,labels,batch_size):
    features = tf.cast(features,tf.float32)    
    dataset = tf.data.Dataset.from_tensor_slices((features,labels))
    
    return dataset.shuffle(1000,seed=42).repeat().batch(batch_size)


In [20]:
def eval_input_fn(features,labels,batch_size):
    features = tf.cast(features,tf.float32)   
    
    if labels is None:
        inputs = features
    else:
        inputs = (features,labels)
    
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    
    return dataset.batch(batch_size)

In [21]:
def create_model(params):
    layers = tf.keras.layers
    return tf.keras.Sequential([
        layers.Dense(params['n_hidden1'],activation=tf.nn.relu),
        layers.Dropout(0.1),
        layers.Dense(params['n_hidden2'],activation=tf.nn.relu),
        layers.Dropout(0.1),
        layers.Dense(params['n_classes'])
    ])


In [22]:
def my_model(features,labels,mode,params):
            
        """DNN with 2 hidden layers , and dropout of 0.1 probability"""
        model = create_model(params)
        
        if mode == tf.estimator.ModeKeys.PREDICT:
            logits = model(features,training=False)
        
            predictions = {
                'class_ids' : tf.argmax(logits,axis=1),
                'probabilities': tf.nn.softmax(logits),
                'logits':logits
            }

            return tf.estimator.EstimatorSpec(mode,predictions=predictions)
    
        if mode == tf.estimator.ModeKeys.TRAIN:      
            logits = model(features,training=True)
            loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,
                                                      logits=logits)
            optimizer = tf.train.AdamOptimizer(learning_rate=0.1)
            
            train_op = optimizer.minimize(loss,
                                     global_step = tf.train.get_global_step())
            
            accuracy = tf.metrics.accuracy(labels=labels,
                                      predictions = tf.argmax(logits,axis=1))
            tf.summary.scalar('accuracy',accuracy[1])
        
            return tf.estimator.EstimatorSpec(mode,loss=loss,train_op=train_op)
            
        if mode == tf.estimator.ModeKeys.EVAL:      
            logits = model(features,training=False)
            loss = tf.losses.sparse_softmax_cross_entropy(labels=labels,
                                                      logits=logits)
            
            accuracy = tf.metrics.accuracy(labels=labels,
                                      predictions = tf.argmax(logits,axis=1))
            metrics = {'accuracy':accuracy}
        
            return tf.estimator.EstimatorSpec(mode,loss=loss,eval_metric_ops = metric)
        
       

In [23]:
config = tf.estimator.RunConfig(tf_random_seed=42)


classifier = tf.estimator.Estimator(
    model_fn=my_model,
    params={
        'n_hidden1': 10,
        'n_hidden2': 10,
        'n_classes': 3
    },
    model_dir = 'my_model',
    config=config
)

INFO:tensorflow:Using config: {'_model_dir': 'my_model', '_tf_random_seed': 42, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002376C364988>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [24]:
batch_size = 100
train_step = 1000

classifier.train(
    input_fn=lambda:train_input_fn(X_train,y_train,batch_size),
    steps=train_step)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into my_model\model.ckpt.
INFO:tensorflow:loss = 1.8516513, step = 0
INFO:tensorflow:global_step/sec: 572.982
INFO:tensorflow:loss = 0.28377807, step = 100 (0.177 sec)
INFO:tensorflow:global_step/sec: 1222.77
INFO:tensorflow:loss = 0.2356748, step = 200 (0.082 sec)
INFO:tensorflow:global_step/sec: 1139.41
INFO:tensorflow:loss = 0.100969724, step = 300 (0.087 sec)
INFO:tensorflow:global_step/sec: 1253.34
INFO:tensorflow:loss = 0.20656008, step = 400 (0.080 sec)
INFO:tensorflow:global_step/sec: 1222.78
INFO:tensorflow:loss = 0.17643635, step = 500 (0.082 sec)
INFO:tensorflow:global_step/sec: 1237.87
INFO:tensorflow:loss = 0.24931479, step = 600 (0.081 sec)
INFO:tensorflow:global_step/sec: 1269.21
INFO:tensorflow:lo